In [2]:
!pip install tsfresh

In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, cv, Pool, CatBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tsfresh import extract_features
from tqdm.auto import tqdm, trange
from sklearn.linear_model import RidgeClassifier, LogisticRegression, LogisticRegressionCV
import random

In [4]:
data = pd.read_csv('dataset.csv',parse_dates=['client_start_date','partnerrolestart_date'])
simple_sub = pd.read_csv('submission_example.csv')

In [5]:
def get_labels(data):
    date = pd.to_datetime('2020-9-01')
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    labels['score'] = labels['client_start_date'].apply(lambda x: 0 if x >= date else 1)
    labels['label_count'] = data.groupby('clientbankpartner_pin')['client_start_date'].agg(lambda x: len([i for i in x.tolist() if i >= date]) )
    return labels

labels = get_labels(data)

In [6]:
train_data = data.copy()
train_data = train_data[train_data['client_start_date'] < pd.to_datetime('2020-9-01')]

In [7]:
def distance_calc(x):
    dist = []
    for i in range(len(x)-1):
        dist += [x[i+1] - x[i]]
    return dist

def create_time_features(df,report_date = '2020-12-01'):
    df['month'] = df['client_start_date'].dt.month
    df['day'] = df['client_start_date'].dt.day
    df['week'] = df['client_start_date'].dt.day.map(lambda x: x // 7)
    df['minute'] = df['client_start_date'].dt.minute
    df['year'] = df['client_start_date'].dt.year

    df['dayofweek'] = df['client_start_date'].dt.dayofweek
    df['all_year_time'] = df['client_start_date'].apply(lambda x: (x.year - 2019) * 365 + x.month * 30 + x.day)
    df['cl_is_weekend'] = (df['dayofweek'] > 4).astype('int16')
    
    df['time_to_report'] = pd.to_datetime(report_date) - df['client_start_date']
    df['days_to_report'] = df['time_to_report'].dt.days.astype(int)
    
    return df

def get_time_features(df):
    time_day_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(['mean', 'std', 'min', 'max', 'median','count'])
    time_day_features.columns = [f'time_day_features_{c}' for c in time_day_features.columns]
    time_day_features['time_day_features_diff'] = time_day_features['time_day_features_max'] - time_day_features['time_day_features_min']
    time_day_features['time_day_features_diversity'] = time_day_features['time_day_features_diff'] / time_day_features['time_day_features_count']
    
    diff_features = df.groupby('clientbankpartner_pin')['days_to_report'].agg(lambda x: distance_calc(x.tolist()))
    time_day_features['max_diff'] = diff_features.apply(lambda x: max(x) if x != []  else -100)
    time_day_features['min_diff'] = diff_features.apply(lambda x: min(x) if x != []  else -100)
    time_day_features['mean_diff'] = diff_features.apply(lambda x: np.mean(x) if x != []  else -100)
    time_day_features['std_diff'] = diff_features.apply(lambda x: np.std(x) if x != []  else -100)
    time_day_features['hard_time_step_polyfit'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[1] if len(x) > 2  else -100)
    time_day_features['hard_time_step_polyfit_st'] = diff_features.map(lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) > 2  else -100)

    return time_day_features

def get_src_code_features(df):
        src = df.groupby('clientbankpartner_pin')['partner_src_type_ccode'].agg(['first'])     
        return src


def partnerrolestart_date_feature(df,report_date = '2020-12-01'):
    df['days_from_start_to_report'] = pd.to_datetime(report_date) - df['partnerrolestart_date']
    start_day_features = df.groupby('clientbankpartner_pin')['days_from_start_to_report'].agg(['min'])
    start_day_features['days_from_start_to_report'] = start_day_features['min'].dt.days
    return start_day_features.drop(['min'],axis=1)

def get_user_features(df):
    user_strs = df.groupby('clientbankpartner_pin')['client_pin'].agg(lambda x:x.tolist())
    user_features = pd.DataFrame()
    user_features['user_nunique'] = user_strs.apply(lambda x: len(np.unique(x)))
    user_features['user_diversity'] = user_features['user_nunique'] / user_strs.apply(len)
    
    user_data = df.groupby('client_pin')['clientbankpartner_pin'].agg(['count'])
    user_data['client_pin'] = user_data.index
    user_data = user_data.rename({'count':'count_tr_per_user'},axis=1)
    user_data.index = range(len(user_data))
    df = df.merge(user_data)
    
    user_features['user_max_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('max')
    user_features['user_mean_act'] =  df.groupby('clientbankpartner_pin')['count_tr_per_user'].agg('mean')
    return user_features


def last_count(df,bins=[5,10,15,20,25,30,35,45,50,60]):
    feature_df = pd.DataFrame(index=df.groupby('clientbankpartner_pin')['client_pin'].agg('count').index)
    for x in bins:
        feature_df[f'count_{x}_prev'] = df[df['days_to_report'] > x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'count_{x}_last'] = df[df['days_to_report'] <= x].groupby('clientbankpartner_pin')['client_pin'].agg('count')
        feature_df[f'percent_{x}'] =  feature_df[f'count_{x}_last'] / (feature_df[f'count_{x}_last'] + feature_df[f'count_{x}_prev'])
        
    return feature_df.fillna(-1)

def create_tsfresh_features(df):
    df_features = extract_features(train_data,column_id='clientbankpartner_pin',column_value='days_to_report')
    return df_features

def compute_all_features(data,report_date):
    data = create_time_features(data,report_date)
    time_features = get_time_features(data)
    src_features = get_src_code_features(data)
    start_day_features = partnerrolestart_date_feature(data,report_date)
    user_features = get_user_features(data)
    last_count_feats = last_count(data)
    
    all_data = pd.concat([time_features,
                      src_features,
                      start_day_features,
                      last_count_feats,
                      user_features,
                      ],axis=1)
    return all_data

In [8]:
train_feats = compute_all_features(train_data,'2020-9-01')

In [9]:
#train_tsmix_feats = extract_features(train_data,column_id='clientbankpartner_pin',column_value='days_to_report')
#train_tsmix_feats.to_parquet('train_tsmixer.parquet')
train_tsmix_feats = pd.read_parquet('train_tsmixer.parquet')

In [10]:
train_embeds = pd.read_parquet('train_embedsV2.parquet')

In [11]:
all_train_data = pd.concat([train_feats.sort_index(),
                            train_tsmix_feats.sort_index(),
                            train_embeds.sort_index(),
                            labels.loc[train_feats.index]],axis=1)

In [12]:
all_train_data = all_train_data.T.drop_duplicates().T

In [13]:
def get_val_splits(data,num_splits=10,n_val=1000,random_seed=56):
    date = pd.to_datetime('2020-6-01')
    labels = data.groupby('clientbankpartner_pin')[['client_start_date']].agg('max')
    good_partners = labels[labels['client_start_date'] >= date].index.tolist()
    all_partners = labels.index.tolist()
    train_idxs, val_idxs = [],[]
    for i in range(num_splits):
        random.seed(random_seed+i)
        random.shuffle(good_partners)
        val_part = good_partners[:n_val]
        tr_part = [part for part in all_partners if part not in val_part]
        train_idxs.append(tr_part)
        val_idxs.append(val_part)
    return train_idxs, val_idxs

train_splits, eval_splits = get_val_splits(train_data)

In [14]:
params = {'iterations':1_000,
          'loss_function':'CrossEntropy',
          'learning_rate':0.01,
          'task_type':'CPU',
          'min_child_samples':20,
          'max_depth':5,
          'eval_metric':'AUC',
          'random_seed':56}

drop_cols = ['label_count','score','client_start_date']

In [ ]:
models = []
scores = []
for i in trange(len(train_splits)):
    train_pool = Pool(data=all_train_data.loc[train_splits[i]].drop(drop_cols,axis=1),
                      label=all_train_data.loc[train_splits[i]]['score'],
                      cat_features=['first'])
    eval_pool = Pool(data=all_train_data.loc[eval_splits[i]].drop(drop_cols,axis=1),
                      label=all_train_data.loc[eval_splits[i]]['score'],
                      cat_features=['first'])
    
    cbm = CatBoostClassifier(**params)
    cbm.fit(train_pool,eval_set=eval_pool,verbose=300)
    scores.append(cbm.get_best_score())
    models.append(cbm)

  0%|          | 0/10 [00:00<?, ?it/s]

0:	test: 0.6897105	best: 0.6897105 (0)	total: 187ms	remaining: 3m 6s
300:	test: 0.7584419	best: 0.7585142 (295)	total: 26.1s	remaining: 1m
600:	test: 0.7610050	best: 0.7610880 (592)	total: 48.8s	remaining: 32.4s
900:	test: 0.7627357	best: 0.7627357 (900)	total: 1m 10s	remaining: 7.77s
999:	test: 0.7631444	best: 0.7632083 (996)	total: 1m 18s	remaining: 0us

bestTest = 0.7632083144
bestIteration = 996

Shrink model to first 997 iterations.
0:	test: 0.7266329	best: 0.7266329 (0)	total: 104ms	remaining: 1m 43s
300:	test: 0.7955820	best: 0.7955820 (300)	total: 23.8s	remaining: 55.3s
600:	test: 0.7975887	best: 0.7976136 (599)	total: 46.3s	remaining: 30.7s
900:	test: 0.7987356	best: 0.7989230 (875)	total: 1m 8s	remaining: 7.54s
999:	test: 0.7987481	best: 0.7989480 (976)	total: 1m 15s	remaining: 0us

bestTest = 0.7989479557
bestIteration = 976

Shrink model to first 977 iterations.
0:	test: 0.7139789	best: 0.7139789 (0)	total: 108ms	remaining: 1m 47s
300:	test: 0.7730597	best: 0.7730638 (299)	

In [ ]:
np.mean([x['validation']['AUC'] for x in scores])

In [ ]:
imp = models[0].get_feature_importance(prettified=True).set_index('Feature Id')
for i in range(1,10):
    imp += models[i].get_feature_importance(prettified=True).set_index('Feature Id')

In [ ]:
imp['is_emb'] = list(map(lambda x: 'emb' in x, imp.index.tolist()))
imp = imp[~imp['is_emb']]

In [ ]:
best_features = imp.sort_values(by='Importances')[::-1][:64].index.tolist() +\
                [f'emb_000{i}' for i in range(10)] + [f'emb_000{i}_tta' for i in range(10)] +\
                [f'emb_00{i}' for i in range(10,32)] + [f'emb_00{i}_tta' for i in range(10,32)]

In [ ]:
imp.sort_values(by='Importances')[::-1].head(30)

In [ ]:
params = {'iterations':2_000,
          'loss_function':'CrossEntropy',
          'learning_rate':0.01,
          'task_type':'CPU',
          'min_child_samples':20,
          'max_depth':4,
          'eval_metric':'AUC',
          'random_seed':56}

In [ ]:
models = []
scores = []
for i in trange(len(train_splits)):
    train_pool = Pool(data=all_train_data.loc[train_splits[i]][best_features],
                      label=all_train_data.loc[train_splits[i]]['score'],
                      cat_features=['first'])
    eval_pool = Pool(data=all_train_data.loc[eval_splits[i]][best_features],
                      label=all_train_data.loc[eval_splits[i]]['score'],
                      cat_features=['first'])
    
    cbm = CatBoostClassifier(**params)
    cbm.fit(train_pool,eval_set=eval_pool,verbose=300)
    scores.append(cbm.get_best_score())
    models.append(cbm) 

In [ ]:
np.mean([x['validation']['AUC'] for x in scores])

In [ ]:
test_data = data.copy()
test_feats = compute_all_features(test_data,'2020-12-01')

In [ ]:
#test_tsmix_feats = extract_features(test_data,column_id='clientbankpartner_pin',column_value='days_to_report')
#test_tsmix_feats.to_parquet('test_tsmixer.parquet')
test_tsmix_feats = pd.read_parquet('test_tsmixer.parquet')

In [ ]:
test_embeds = pd.read_parquet("test_embedsV2.parquet")

In [ ]:
all_test_data = pd.concat([test_feats.sort_index(),test_tsmix_feats.sort_index(),test_embeds.sort_index()],axis=1)

In [ ]:
simple_sub['clientbankpartner_pin']

In [ ]:
all_test_data_lc = all_test_data.loc[simple_sub['clientbankpartner_pin']]

In [ ]:
test_pool = Pool(all_test_data_lc[best_features],
                 cat_features=['first'])

In [ ]:
scores = np.mean([cbm.predict_proba(test_pool)[:,1] for cbm in models],axis=0)

In [ ]:
simple_sub['score'] = scores

In [136]:
simple_sub.to_csv('CatBoost_Embeds_TTA.csv',index=False)

In [137]:
simple_sub

clientbankpartner_pin     score
0                      6781  0.966306
1                    236905  0.913529
2                    125779  0.764930
3                      1952  0.777171
4                      4872  0.390060
...                     ...       ...
4503                 121120  0.667449
4504                  60667  0.346244
4505                   5065  0.338639
4506                 133125  0.571232
4507                 121963  0.348032

[4508 rows x 2 columns]

In [61]:
pd.read_csv('/kaggle/working/embeds_mix.csv')

clientbankpartner_pin     score
0                      6781  0.957924
1                    236905  0.920736
2                    125779  0.744447
3                      1952  0.756435
4                      4872  0.391438
...                     ...       ...
4503                 121120  0.658698
4504                  60667  0.345487
4505                   5065  0.317884
4506                 133125  0.546246
4507                 121963  0.335644

[4508 rows x 2 columns]